In [ ]:
!pip install datasets transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 468.7/468.7 kB 11.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 54.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.9/132.9 kB 9.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 17.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.2/212.2 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.1/200.1 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 33.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.2/114.2 kB 13.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.8/158.8 kB 19.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 264.6/264.6 kB 29.1 MB/s eta 0:00:00


In [1]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:
import torch
from transformers import AutoTokenizer, AutoModel

# Load the pre-trained BERT model and tokenizer
model_name = 'bert-base-uncased'
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
from datasets import *
dataset = load_dataset("cc_news", split = 'train')

Generating train split:   0%|          | 0/708241 [00:00<?, ? examples/s]

Dataset cc_news downloaded and prepared to /root/.cache/huggingface/datasets/cc_news/plain_text/1.0.0/ae469e556251e6e7e20a789f93803c7de19d0c4311b6854ab072fecb4e401bd6. Subsequent calls will reuse this data.


In [ ]:
def load_corpus_data():
  res = {}
  num = 10000
  i=0
  while len(res)<num:
    des = dataset[i]['description']
    if 64<len(des)<512 and des != '':
      res[des] = i # using dict to dedup
    i+=1
    
  print(len(res))
  return list(res)

corpus = load_corpus_data()

10000


In [ ]:
from random import randint
for i in range(20):
  print(corpus[randint(0,999)])

Looking for your next audition shoe? Shot at and in collaboration with Broadway Dance Center, Só Dança has launched a new collection of shoes working with some pretty famous faces of the musical theater world! Offered in two different styles and either 2.5" or 3" heels, top industry professionals ar...
Soap Manufacturers in the USA - BizVibe Announces a New B2B Networking Platform for the Soap Industry in the USA
Egretia Teams Up with Egret Technologies to Create the World's Most Powerful HTML5 Workflow!
Senate overwhelmingly approves new package of sanctions to punish Russia for meddling in 2016 election
There are a whole slew of entertainment options coming up this week in Taipei.Here is a list of events we think readers might enjoy around Taipei from June 1- June 8.
今天世界迎來了一個重要的日子—「世界曼德拉日」，這位南非著名的反種族隔離鬥士以他的努力，獲得了全世界的敬重，世界於是將他的生日定為節日，以紀念他對於人權的貢獻。
Stratasys Advancing Additive Manufacturing Into Mainstream With New Solutions for Factory Floor
PULASKI – Shae Dunn went 2-for-4 and drove 

In [ ]:
def load_search_queries():
  return [
    "British police",
    "Leighton Town can beat anyone on their day, according to manager Scott Reynolds.",
    "traffic restriction"
  ]

queries = load_search_queries()

In [ ]:
import torch
from tqdm import tqdm
import numpy as np
corpus_embeddings = []

with tqdm(total=len(corpus)) as bar:
  for text in corpus:
    input_ids = torch.tensor(tokenizer.encode(text, add_special_tokens=True)).unsqueeze(0)
    outputs = model(input_ids)
    last_hidden_states = outputs[0]
    avg_pooling = torch.mean(last_hidden_states, dim=1).squeeze()
    corpus_embeddings.append(avg_pooling.detach().numpy().tolist())
    bar.update(1)

corpus_embeddings = np.array(corpus_embeddings)

In [ ]:
# save embeddings
import json
with open('./drive/MyDrive/DenseRetrieve/ccnews_10000_embeddings.json','w') as f:
  d = json.dumps(corpus_embeddings)
  f.write(d)

In [3]:
# load embeddings
import json
import numpy as np
with open('./drive/MyDrive/DenseRetrieve/ccnews_10000_embeddings.json','r') as f:
  d = f.read()
  corpus_embeddings = np.array(json.loads(d))
print(len(corpus_embeddings))

10000


In [ ]:
query_embeddings = []
for text in queries:
  input_ids = torch.tensor(tokenizer.encode(text, add_special_tokens=True)).unsqueeze(0)
  outputs = model(input_ids)
  last_hidden_states = outputs[0]
  avg_pooling = torch.mean(last_hidden_states, dim=1).squeeze()
  query_embeddings.append(avg_pooling.detach().numpy())

In [ ]:
from numpy import dot
from numpy.linalg import norm

for k,query_embedding in enumerate(query_embeddings):
  print(f"query = {queries[k]}")
  relevances = []
  for i,corpus_embedding in enumerate(corpus_embeddings):
    relevances.append(
        (dot(query_embedding, corpus_embedding)/(norm(query_embedding)*norm(corpus_embedding)),i)
    )
  
  sorted_indices = sorted(relevances, key = lambda x: x[0], reverse = True)
  for i in range(5):
    print(corpus[
        sorted_indices[i][1]
    ])
  print('='*20)

query = British police
What's going on in ballet this week? We've pulled together some highlights.
NYCB has announced an interim leadership team to run the company during Peter Martin's leave of absence amidst sexual harassment accusations.
The Philadelphia Eagles and the New England Patriots aren't the only teams bringing Super Bowl entertainment this week. To celebrate game day (and cheer on their region's respective teams), the dancers of Pennsylvania Ballet and Boston Ballet took a break from their usual rehearsals to perform some Super Bowl-themed choreography.
We interviewed New York City Ballet principal Rebecca Krohn on her company retirement, exciting new career as a ballet master, and her advice for young dancers.
Looking for your next audition shoe? Shot at and in collaboration with Broadway Dance Center, Só Dança has launched a new collection of shoes working with some pretty famous faces of the musical theater world! Offered in two different styles and either 2.5" or 3" he